In [1]:
# Initial imports
import os
import json
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation

%matplotlib inline

In [2]:
# Load .env enviroment variables
load_dotenv()

True

In [3]:
# Set Alpaca API key and secret
alpaca_api = os.getenv("ALPACA_API")
alpaca_secret = os.getenv("ALPACA_SECRET")

# Create the Alpaca API object
api = tradeapi.REST(
    alpaca_api,
    alpaca_secret,
    api_version = "v2"
)

In [11]:
# Format current date as ISO format
start_date = pd.Timestamp("2018-12-31", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-03-06", tz="America/New_York").isoformat()

# Set the tickers
tickers = ["FB", "AMZN", "AAPL", "NFLX", "GOOG", "SHOP", "PLD", "TTWO", "ATVI", "NVDA", "AMD",
          "PEP", "MKC", "DPZ", "WING", "COST", "ETSY", "CPB", "OKTA", "ZM"]

# Set timeframe to '1D' for Alpaca API
timeframe = "1D"

# Get price data for 20 picks 
ticker_data = api.get_barset(
    tickers,
    timeframe,
    start=start_date,
    end=end_date,
    limit=1000,
).df

# Drop any rows with NaN
ticker_data.dropna(inplace=True)
ticker_data.isna().sum()


# Preview DataFrame
ticker_data.head(20)



AAPL                                       \
                              open      high      low   close    volume   
time                                                                      
2019-04-18 00:00:00-04:00  203.120  204.1500  202.520  203.86  21928367   
2019-04-22 00:00:00-04:00  202.830  204.9400  202.340  204.64  13720923   
2019-04-23 00:00:00-04:00  204.430  207.7500  203.900  207.51  19401417   
2019-04-24 00:00:00-04:00  207.360  208.4800  207.050  207.18  14914939   
2019-04-25 00:00:00-04:00  206.830  207.7600  205.120  205.24  15908807   
2019-04-26 00:00:00-04:00  204.900  205.0000  202.120  204.29  16315669   
2019-04-29 00:00:00-04:00  204.400  205.9700  203.860  204.61  19641066   
2019-04-30 00:00:00-04:00  203.060  203.4000  199.110  200.57  35362106   
2019-05-01 00:00:00-04:00  209.880  215.3100  209.230  210.52  57751414   
2019-05-02 00:00:00-04:00  209.840  212.6500  208.130  209.17  29014844   
2019-05-03 00:00:00-04:00  210.740  211.8400  210.230  211.78  17987793   
2019-05-06 00:00:00-04:00  204.290  208.8400  203.500  208.60  28949691   
2019-05-07 00:00:00-04:00  205.880  207.4175  200.825  202.86  34328425   
2019-05-08 00:00:00-04:00  201.900  205.3400  201.750  202.90  22729670   
2019-05-09 00:00:00-04:00  200.400  201.6800  196.660  200.72  32427147   
2019-05-10 00:00:00-04:00  197.419  198.8500  192.770  197.30  36118438   
2019-05-13 00:00:00-04:00  187.710  189.4800  182.850  185.72  50491792   
2019-05-14 00:00:00-04:00  186.410  189.7000  185.410  188.65  31498214   
2019-05-15 00:00:00-04:00  186.270  191.7500  186.020  190.92  23655393   
2019-05-16 00:00:00-04:00  189.910  192.4689  188.840  190.14  28660083   

                             AMD                                        ...  \
                            open     high      low    close     volume  ...   
time                                                                    ...   
2019-04-18 00:00:00-04:00  27.60  27.8800  27.3400  27.6767   37302337  ...   
2019-04-22 00:00:00-04:00  27.62  28.2300  27.3900  28.1700   33846781  ...   
2019-04-23 00:00:00-04:00  28.18  28.4900  27.7900  27.9700   37974538  ...   
2019-04-24 00:00:00-04:00  28.10  28.8500  27.9300  28.4600   48395877  ...   
2019-04-25 00:00:00-04:00  28.65  28.8600  27.3600  27.6500   51487295  ...   
2019-04-26 00:00:00-04:00  27.62  27.9000  27.0500  27.8800   44699014  ...   
2019-04-29 00:00:00-04:00  27.90  28.1400  27.5000  27.6811   40385475  ...   
2019-04-30 00:00:00-04:00  27.59  27.8000  26.9400  27.6300   54019126  ...   
2019-05-01 00:00:00-04:00  28.95  29.1500  26.7800  26.8000  127287384  ...   
2019-05-02 00:00:00-04:00  26.94  28.6400  26.6100  28.2900   95312594  ...   
2019-05-03 00:00:00-04:00  28.30  28.4201  27.6600  28.2200   51718501  ...   
2019-05-06 00:00:00-04:00  26.72  27.5000  26.4500  27.4300   64760337  ...   
2019-05-07 00:00:00-04:00  27.20  27.3500  26.2100  26.6600   70007742  ...   
2019-05-08 00:00:00-04:00  26.41  27.7100  26.2700  27.0900   62428679  ...   
2019-05-09 00:00:00-04:00  26.70  27.3800  26.0300  27.2000   69825507  ...   
2019-05-10 00:00:00-04:00  27.03  28.1000  26.9300  27.9500   79121383  ...   
2019-05-13 00:00:00-04:00  26.98  27.2300  26.1000  26.2400   92488651  ...   
2019-05-14 00:00:00-04:00  26.53  27.4800  26.1500  27.3100   76955796  ...   
2019-05-15 00:00:00-04:00  26.87  27.7900  26.7300  27.5800   52483356  ...   
2019-05-16 00:00:00-04:00  27.37  28.3700  27.2699  28.0200   60976490  ...   

                            WING                                       ZM  \
                            open     high    low  close     volume   open   
time                                                                        
2019-04-18 00:00:00-04:00  76.08  78.4500  75.90  77.95   318224.0  65.00   
2019-04-22 00:00:00-04:00  77.53  78.8700  76.30  77.04   318655.0  61.00   
2019-04-23 00:00:00-04:00  77.05  78.4700  76.53  77.22   362748.0  66.87   
2019-04-24 00:00:00-04:0

In [12]:
# Create baseline dataframe from 2019 dataset
baseline_df = ticker_data.iloc[:253]
baseline_df

# Create closing prices Data frame from 2019 baseline data
closing_df = pd.DataFrame()

closing_df["AAPL"] = baseline_df["AAPL"]["close"]
closing_df["AMD"] = baseline_df["AMD"]["close"]
closing_df["AMZN"] = baseline_df["AMZN"]["close"]
closing_df["ATVI"] = baseline_df["ATVI"]["close"]
closing_df["COST"] = baseline_df["COST"]["close"]
closing_df["CPB"] = baseline_df["CPB"]["close"]
closing_df["DPZ"] = baseline_df["DPZ"]["close"]
closing_df["ETSY"] = baseline_df["ETSY"]["close"]
closing_df["FB"] = baseline_df["FB"]["close"]
closing_df["GOOG"] = baseline_df["GOOG"]["close"]
closing_df["MKC"] = baseline_df["MKC"]["close"]
closing_df["NFLX"] = baseline_df["NFLX"]["close"]
closing_df["NVDA"] = baseline_df["NVDA"]["close"]
closing_df["SHOP"] = baseline_df["SHOP"]["close"]
closing_df["PEP"] = baseline_df["PEP"]["close"]
closing_df["PLD"] = baseline_df["PLD"]["close"]
closing_df["TTWO"] = baseline_df["TTWO"]["close"]
closing_df["WING"] = baseline_df["WING"]["close"]
closing_df["ZM"] = baseline_df["ZM"]["close"]

# Drop the time component of the date
closing_df.index = closing_df.index.date
closing_df
# Compute daily returns
#df_daily_returns = df_closing_prices.pct_change().dropna()

# Display sample data
#df_daily_returns.head(10)


,AAPL,AMD,AMZN,ATVI,COST,CPB,DPZ,ETSY,FB,GOOG,MKC,NFLX,NVDA,SHOP,PEP,PLD,TTWO,WING,ZM
2019-04-18,203.86,27.6767,1861.40,45.30,245.86,39.390,267.60,63.40,178.3200,1236.81,154.37,360.35,186.30,220.7800,127.13,73.590,90.74,77.95,62.4200
2019-04-22,204.64,28.1700,1888.04,45.34,243.44,39.330,266.09,63.53,181.4900,1249.09,152.67,377.36,188.45,224.3000,126.44,72.930,91.92,77.04,65.6300
2019-04-23,207.51,27.9700,1924.17,46.50,245.58,38.910,270.25,65.23,183.7700,1264.32,151.28,381.88,190.67,223.4000,127.09,74.210,93.29,77.22,69.0500
2019-04-24,207.18,28.4600,1900.88,46.76,247.97,38.990,283.20,66.07,182.6300,1255.45,151.33,374.19,191.17,219.8000,126.95,75.310,93.63,77.76,63.2000
2019-04-25,205.24,27.6500,1902.16,47.41,245.54,38.165,276.43,67.86,193.1705,1262.98,150.97,368.34,186.84,219.0900,126.09,75.440,92.90,77.74,65.0200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-15,284.45,55.0000,2309.27,65.67,310.27,50.500,354.10,56.75,176.9100,1262.47,152.49,426.75,280.85,497.1400,135.09,86.495,123.50,101.45,151.5000
2020-04-16,286.72,56.9100,2408.19,68.07,321.42,51.110,360.46,59.68,176.2800,1263.62,155.43,439.17,294.79,526.4300,135.71,88.120,126.85,102.96,150.3900
2020-04-17,282.74,56.5800,2372.87,66.89,317.88,50.340,362.86,61.06,179.2500,1283.23,156.99,423.05,292.32,591.5296,137.56,90.250,124.46,107.14,149.9800
2020-04-20,276.85,56.9200,2393.08,66.51,311.79,51.290,368.73,65.04,178.2600,1267.58,156.13,437.38,286.75,630.1587,134.52,88.610,125.36,109.35,148.9899
